## Confusion matrix for Grant Irrigated

  - In a meeting (May 19th) we decided to drop non-irrigated fields
  - I made a mistake for Grant 2018 and did not filter out non-irrigated fields. Hence, we can create statistics related to non-irrigated part. However, we did toss the non-irrigated fields of 2018 for other parts.
  - If Min's code is fast, then we do not need to drop anything. I will just run everything. 

In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import os, os.path
import itertools
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc

# sys.path.append('/home/hnoorazar/remote_sensing_codes/')
# import remote_sensing_core as rcp

In [3]:
param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

double_crop_potens = pd.read_csv(param_dir + "double_crop_potential_plants.csv")

#### Read ShapeFile Data

In [4]:
shapeFile_Data_dir = "/Users/hn/Documents/01_research_data/" + \
                      "remote_sensing/01_Data_part_not_filtered/"

WSDA_DataTable_2018 = pd.read_csv(shapeFile_Data_dir + "WSDA_DataTable_2018.csv")

WSDA_DataTable_2018 = WSDA_DataTable_2018[WSDA_DataTable_2018.county == "Grant"]

WSDA_DataTable_2018["DataSrc"] = WSDA_DataTable_2018["DataSrc"].str.lower()
WSDA_DataTable_2018["CropTyp"] = WSDA_DataTable_2018["CropTyp"].str.lower()

print(WSDA_DataTable_2018.county.unique())
print(WSDA_DataTable_2018.DataSrc.unique())

['Grant']
['nass' 'wsda' 'producer' 'conservation district']


In [5]:
indeks = "EVI"

peak_tables_dir = "/Users/hn/Documents/01_research_data/" + \
                  "remote_sensing/01_NDVI_TS/04_Irrigated_eastern_Cloud70/" + \
                  "Grant_2018_irrigated/savitzky_" + indeks + "/Grant_Irrigated_2018_no_plot/"

####
####  parameters
####
deltas = [0.1, 0.2, 0.3, 0.4]
windows_degrees = [(3, 1), (3, 2),
                   (5, 1), (5, 2), (5, 3),
                   (7, 1), (7, 2), (7, 3), 
                   (9, 1), (9, 2), (9, 3)]

windows_degrees = [[3, 1], [3, 2],
                   [5, 1], [5, 2], [5, 3],
                   [7, 1], [7, 2], [7, 3], 
                   [9, 1], [9, 2], [9, 3]]



delta_windows_degrees = [[i, j] for i in deltas for j in windows_degrees]

output_columns = ['parameters', 'actual_2_pred_2', 'actual_2_pred_Not2',
                  'actual_Not2_pred_2', 'actual_Not2_pred_Not2',]

### Generate different datafamres based on the following variables

- 2 * 2 * 2 * 2 different combinations of NASS, Double_by_Notes, Irrigated, LastSurveyYear

For now, leave out the fucking last survey year!

In the following we have abbreviated:  
 - AF: All Fields
 - DP: Double Potential Fields (i.e. perennials out)
 - Irr: Just Irrigated Fields
 - BothIrr: both irrigated and non-irrigated

In [6]:
# NASS_out = False
# non_Irr_out = True
# perennials_out = False
exactly_2_peaks = True

for NASS_out in [False]: # True,
    for non_Irr_out in [True]: # we have dropped out non-irrigated fields in the peak_finding step
        for perennials_out in [True]: # , False
            #### 
            #### build output dataframe
            #### 
            output = pd.DataFrame(data=None, 
                                  index = np.arange(len(delta_windows_degrees)), # delta_windows_degrees, # 
                                  columns = output_columns)
            output['parameters'] = delta_windows_degrees
            
            #### 
            #### Build shapeFile info accordingly
            #### 
            curr_SF = WSDA_DataTable_2018.copy()
            
            if perennials_out == True:
                curr_SF = curr_SF[curr_SF.CropTyp.isin(double_crop_potens['Crop_Type'])]
                Pere_name = "PereOut_"
            else:
                Pere_name = "PereIn_"
            
#             print("perennials_out " + str(perennials_out))
#             print (curr_SF.CropTyp.unique())
            
            if NASS_out == True:
                curr_SF = rc.filter_out_NASS(curr_SF)
                NASS_name = "NASSOut_"
            else:
                NASS_name = "NASSin_"
                
            if non_Irr_out == True:
                curr_SF = rc.filter_out_nonIrrigated(curr_SF)
                non_Irr_name = "JustIrr"
            else:
                non_Irr_name = "BothIrr"
 
            # print("_____________________________________________")
            # print("non_Irr_out " + str(non_Irr_out) + "curr_SF.Irrigtn:")
            # print (curr_SF.Irrigtn.unique())
            
            print ("NASS_out: " + str(NASS_out) + ", non_Irr_out: " + str(non_Irr_out) + \
                   ", perennials_out: " + str(perennials_out))

            for location, params in enumerate(output['parameters']):
#                 if location % 15 ==0:
#                     print (location)
                delt = params[0]
                win_deg = params[1]
                window = win_deg[0]
                degree = win_deg[1]
                doubl_pk_file = peak_tables_dir + "delta" + str(delt) + "_Sav_win" + str(window) + \
                                "_Order" + str(degree) + "/all_poly_and_maxs_savitzky.csv"

                doubl_peak_table = pd.read_csv( doubl_pk_file, low_memory=False)
                
                doubl_peak_table["CropTyp"] = doubl_peak_table["CropTyp"].str.lower()
                ############################################################
                #
                # Drop last row that is empty. We will fix this in the peak_finding step
                # Drop the last row
                #
                ############################################################
                last_row_id = doubl_peak_table.shape[0] - 1
                doubl_peak_table = doubl_peak_table.drop([last_row_id], axis=0)
                
                if perennials_out == True:
                    doubl_peak_table = doubl_peak_table[\
                                                    doubl_peak_table.CropTyp.isin(double_crop_potens['Crop_Type'])]
#                 print("perennials_out " + str(perennials_out))
#                 print (doubl_peak_table.CropTyp.unique())
            
#                   print ("No. of rows after dropping perennials \
#                          are [%(nrow)d]." % {"nrow":doubl_peak_table.shape[0]})
                            
                if NASS_out == True:
                    doubl_peak_table = rc.filter_out_NASS(doubl_peak_table)

#                 print ("NASS_out: " + str(NASS_out))
#                 print ("curr_SF.DataSrc.unique(): ")
#                 print (curr_SF.DataSrc.unique())
                
#                 print ("doubl_peak_table.DataSrc.unique(): ")
#                 print (doubl_peak_table.DataSrc.unique())
#                 print("____________________________________________________")

                if non_Irr_out == True:
                    doubl_peak_table = rc.filter_out_nonIrrigated(doubl_peak_table)
                # print("_____________________________________________")
                # print("non_Irr_out " + str(non_Irr_out) + " doubl_peak_table.Irrigtn:")
                # print (doubl_peak_table.Irrigtn.unique())

                # print ("No. of rows are [%(nrow)d]." % {"nrow":doubl_peak_table.shape[0]})
                doubl_peak_table.drop(['max_Doy', 'max_value'], axis=1, inplace=True)
                doubl_peak_table.drop_duplicates(inplace=True)
                # print ("No. of rows after dropping dupliates are [%(nrow)d]." % \
                # {"nrow":doubl_peak_table.shape[0]})

                #### 
                #### Populate output dataframe
                #### 
                actual_double_cropped = rc.filter_double_by_Notes(curr_SF)
                actual_Notdouble_cropped = rc.filter_Notdouble_by_Notes(curr_SF)
                
                if exactly_2_peaks == False:
                    predicted_double_peaks = doubl_peak_table[doubl_peak_table.max_count >= 2].copy()
                    predicted_Notdouble_peaks = doubl_peak_table[doubl_peak_table.max_count < 2].copy()
                    exactly_2_peaks_name = "morethan_2peaks"
                else: 
                    predicted_double_peaks = doubl_peak_table[doubl_peak_table.max_count == 2].copy()
                    predicted_Notdouble_peaks = doubl_peak_table[doubl_peak_table.max_count != 2].copy()
                    exactly_2_peaks_name = "exactly_2_peaks"

                # print ("There are [%(nrow)d] IDs in curr_SF." % \
                #       {"nrow":len(curr_SF['ID'])})

                # print ("of which [%(nrow)d] are unique." % \
                #        {"nrow":len(curr_SF['ID'].unique())})

                actual_2_pred_2 = sum(actual_double_cropped['ID'].isin(predicted_double_peaks['ID']))
                actual_Not2_pred_2 = sum(actual_Notdouble_cropped['ID'].isin(predicted_double_peaks['ID']))

                # the follwoing two lines would not work, since we have not 
                # included the fields with no maximum in our output
                # actual_2_pred_Not2 = sum(actual_double_cropped['ID'].isin(predicted_Notdouble_peaks['ID']))
                # actual_Not2_pred_Not2 = sum(actual_Notdouble_cropped['ID'].isin(predicted_Notdouble_peaks['ID']))

                actual_2_pred_Not2 = len(actual_double_cropped['ID'].unique()) - actual_2_pred_2
                actual_Not2_pred_Not2 = len(actual_Notdouble_cropped['ID'].unique()) - actual_Not2_pred_2

                fillin_col = ["actual_2_pred_2", "actual_2_pred_Not2", "actual_Not2_pred_2", "actual_Not2_pred_Not2"]
                fillin_vals = [actual_2_pred_2, actual_2_pred_Not2, actual_Not2_pred_2, actual_Not2_pred_Not2]
                output.loc[location, fillin_col] = fillin_vals
            
            
            ###########
            output['parameters'] = output['parameters'].astype("str")
            write_path = "/Users/hn/Documents/01_research_data/" + \
                         "remote_sensing/01_NDVI_TS/" + \
                         "04_Irrigated_eastern_Cloud70/" + \
                         "Grant_2018_irrigated/savitzky_" + indeks + "/"
            
            os.makedirs(write_path, exist_ok=True)
            filename = write_path + "saviztky_" + indeks + "_" + Pere_name + \
                       NASS_name + non_Irr_name + "_confusion_" + exactly_2_peaks_name + ".csv"
            output.to_csv(filename, index = False)
            del(output)

NASS_out: False, non_Irr_out: True, perennials_out: True


#### 1. How many of the fields coded with double in Notes are double peakes?

#### 2. How many of the ** <ins>irrigated</ins> ** fields coded with double in Notes are double peakes?

#### 3. How many of the fields coded with double ( ** <ins> by sources other than NASS</ins> **)  in Notes are double peakes?

#### 4. How many of the ** <ins> irrigated fields </ins>  ** coded with double (** <ins> by sources other than NASS</ins> **) in Notes are double peakes?